Este colab fue desarrollado por Arnold Charry Armero.

# Regresión Lineal Múltiple

Una vez se cuestiona la relación lineal de una sola característica con una variable de respuesta, surge la regresión múltiple. En esta la hipótesis nula es que los coeficientes de correlación $  \hat{\beta_{p}} $ son iguales a 0 y la hipótesis alternativa es que existe al menos una relación con la variable dependiente. Esto se puede ver en las siguientes expresiones matemáticas:

$$  H_{0}:\hat{\beta_{1} }= \hat{\beta_{2} } = \cdots = \hat{\beta_{p} } = 0 $$

$$  H_{1}: \mathrm{Al \:menos \: un \:} \:  \hat{\beta_{p}}\:\mathrm{no  \:es \:0} $$

Es importante destacar que se deben evaluar cada una de las variables $ \hat{\beta_{p}} $, ya que alguna puede ser descartada si su valor-p es mayor que el nivel de significancia. Esto se hace para obtener un modelo más sencillo y fácil de comprender para el usuario. A partir de varias características, se desea dar un peso que vienen siendo los coeficientes $ \hat{\beta_{p}} $ que expliquen el comportamiento de una variable dependiente determinada. Matemáticamente la regresión lineal múltiple se ve de la siguiente manera:

$$ \hat{y} = \hat{\beta_{0}}+ \hat{\beta _{1}} x_{1} + \hat{\beta _{2}} x_{2} + \cdots + \hat{\beta _{p}} x_{p} $$

Que genéralizandolo queda de la siguiente manera,

$$ \hat{y} = \hat{\beta_{0}}+\sum_{i=1}^{n}\hat{\beta _{i}}\:  x_{i} $$

Donde $ \hat{\beta_{0}} $ es una constante y $ \hat{\beta _{p}} $ es la pendiente de la característica $p$; ambas se conocen como parámetros o coeficientes del modelo matemático. Mientras que, por otro lado, $ \hat{y} $ es la predicción que realiza el modelo. Para obtener los parámetros iniciales se debe de tomar una muestra de datos y optimizar los mínimos cuadrados.

Los mínimos cuadrados son una técnica de medición para minimizar el error presente del modelo que se reflejan de la siguiente manera,

$$ e_{i}^{2} = (y_{i} - \hat{y_{i}})^{2}$$

Se debe minimizar el error, el cual se define como,

$$ \text{min} \sum_{i=1}^{n} e_{i}^{2} $$

$$ \text{min} \sum_{i=1}^{n} (y_{i} - \hat{\beta_{0}}+ \hat{\beta _{1}} x_{i1} + \hat{\beta _{2}} x_{i2} + \cdots + \hat{\beta _{p}} x_{ip})^{2} $$

Ahora se sigue con la implementación en código,

In [ ]:
# Se importan las librerias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, cross_val_score

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Para la Regresión Lineal Múltiple cuando se tienen variables categóricas, se deben de crear $ n - 1 $ variables que correspondan a las categorías que se van a representar. Para ello, se deben crear variables dummy para las variables categóricas.

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Machine Learning/Bases de Datos/Student_Performance.csv')

In [ ]:
df.head()

,Hours Studied,Previous Scores,Extracurricular Activities,Sleep Hours,Sample Question Papers Practiced,Performance Index
0,7,99,Yes,9,1,91.0
1,4,82,No,4,2,65.0
2,8,51,Yes,7,2,45.0
3,5,52,Yes,5,2,36.0
4,7,75,No,8,5,66.0


In [ ]:
# Obtenemos las características
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

## Preprocesamiento de Datos

In [ ]:
# Se detectan las columnas categóricas
cat_cols = df.select_dtypes(include=['object', 'category']).columns
cat_indices = [df.columns.get_loc(col) for col in cat_cols]

# Se detectan las columnas numéricas
num_indices = [i for i in range(df.shape[1] - 1) if i not in cat_indices]


# Se crea el transformador
ct = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_indices),
        ('encoder', OneHotEncoder(drop='first', sparse_output=False, dtype=int), cat_indices)], remainder='passthrough')

## Separación en Base de datos de Entrenamiento y Prueba

In [ ]:
# Se divide la base de datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

## Escalado de Datos

In [ ]:
# Se escalan las variables
X_train = ct.fit_transform(X_train)
X_test = ct.transform(X_test)

In [ ]:
# Visualizar el array X_train
print(X_train)

[[ 0.01177784  1.46795645 -0.31740964 -0.20579059  0.        ]
 [-1.14669864  1.63989277  0.2720233  -0.20579059  0.        ]
 [-1.14669864 -0.25140672  0.86145625 -1.25196174  1.        ]
 ...
 [-1.53285746  0.37902645 -0.31740964  0.14293313  1.        ]
 [-0.37438098 -1.45496093 -1.49627552  1.18910429  0.        ]
 [ 1.17025433  0.78021119  0.86145625 -1.25196174  1.        ]]


## Entrenamiento del Modelo

In [ ]:
# Entrenamos el modelo
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

Obteniendo los coeficientes del modelo,

In [ ]:
print('Coeficientes: ', model.coef_)
print('Intercepción: ', model.intercept_)

Coeficientes:  [ 7.38534048 17.77751241  0.80896388  0.53843878  0.66530347]
Intercepción:  54.772723776401556


Que convirtiéndolos a las variables originales sería,

In [ ]:
# Se accede al escalador
scaler = ct.named_transformers_.get('num')

# Desescalado
n_num = len(num_indices)
coef_scaled = model.coef_

coef_num_scaled = coef_scaled[:n_num]
coef_cat = coef_scaled[n_num:]

coef_num_original = coef_num_scaled / scaler.scale_
intercept_original = model.intercept_ - np.sum(scaler.mean_ * coef_num_scaled / scaler.scale_)

# Nombres de variables
num_names = df.columns[num_indices]
cat_encoder = ct.named_transformers_['encoder']
cat_feature_names = cat_encoder.get_feature_names_out(cat_cols)

feature_names = list(num_names) + list(cat_feature_names)
coef_original = np.concatenate([coef_num_original, coef_cat])

# Tabla final
coef_table = pd.DataFrame({'Variable': feature_names, 'Coeficiente (original)': coef_original})

print('\n--- Coeficientes en escala original ---')
print(coef_table)
print('\nIntercepto en escala original:', intercept_original)



--- Coeficientes en escala original ---
                           Variable  Coeficiente (original)
0                     Hours Studied                2.851914
1                   Previous Scores                1.018867
2                       Sleep Hours                0.476830
3  Sample Question Papers Practiced                0.187766
4    Extracurricular Activities_Yes                0.665303

Intercepto en escala original: -34.07520822521332


Que, se debe tomar en cuenta, que el último coeficiente corresponde a la variable dummy o binaria por el orden en el que se hizo el transformador.

Realizando una predicción,

In [ ]:
# Se debe de hacer en el orden original
print("Predicción:", model.predict(ct.transform([[8, 90, 'Yes', 9, 3]]))[0])

Predicción: 85.95817955695779


In [ ]:
# Obtenemos las predicciones
y_pred = model.predict(X_test)
print(y_pred.reshape(len(y_pred),1))

[[50.45128503]
 [53.09379171]
 [78.24502652]
 ...
 [64.56863194]
 [25.89718776]
 [18.82598463]]


## Rendimiento del Modelo

In [ ]:
# KPI's del Modelo
MAE = mean_absolute_error(y_test, y_pred)
print('MAE: {:0.2f}%'.format(MAE / np.mean(y_test) * 100))
MSE = mean_squared_error(y_test, y_pred)
RMSE = np.sqrt(MSE)
print('RMSE: {:0.2f}%'.format(RMSE / np.mean(y_test) * 100))
r2 = r2_score(y_test, y_pred)
print('R2: {:0.2f}'.format(r2))

MAE: 2.91%
RMSE: 3.64%
R2: 0.99


## Eliminación hacia Atrás

Por medio del método de eliminación hacia atrás, se descartan las variables que no sean estadísticamente relevantes. Un modelo no por tener más variables explica mejor un fenómeno, por lo que se busca que las variables sean relevantes y estadísticamente significativas en la explicación. Por medio de la función *Ordinary Least Squares* del módulo StatsModels se identifica cuales variables no son estadísticamente relevantes.

In [ ]:
# Se convierte el vector X en float
X_train = X_train.astype(float)

In [ ]:
# Se crea una función que automatice la eliminación hacia atrás

def backward_elimination(X, y, SL=0.05):
    # Se añade columna de 1s para el intercepto
    X = np.append(arr=np.ones((X.shape[0], 1)).astype(int), values=X, axis=1)
    num_vars = X.shape[1]
    selected_vars = list(range(num_vars))

    while True:
        X_opt = X[:, selected_vars]
        model = sm.OLS(y, X_opt).fit()
        p_values = model.pvalues

        # Se encuentra el p-valor máximo
        max_p_value = max(p_values)
        if max_p_value > SL:
            max_p_index = p_values.argmax()
            removed_var = selected_vars[max_p_index]
            print(f"Eliminando variable en índice {removed_var} con p-valor {max_p_value:.4f}")
            selected_vars.pop(max_p_index)
        else:
            break

    print("\nVariables seleccionadas (índices):", selected_vars)
    return X[:, selected_vars], selected_vars, model

In [ ]:
# Se ven los resultados
X_selected, selected_indices, final_model = backward_elimination(X_train, y_train, SL=0.05)

# Se ve el resumen del modelo
print(final_model.summary())


Variables seleccionadas (índices): [0, 1, 2, 3, 4, 5]
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.989
Model:                            OLS   Adj. R-squared:                  0.989
Method:                 Least Squares   F-statistic:                 1.425e+05
Date:                Thu, 16 Oct 2025   Prob (F-statistic):               0.00
Time:                        17:15:07   Log-Likelihood:                -17058.
No. Observations:                8000   AIC:                         3.413e+04
Df Residuals:                    7994   BIC:                         3.417e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------

## Validación Cruzada

In [ ]:
# Aplicar K-fold Cross Validation
scores = cross_val_score(estimator = model, X = X_train, y = y_train, cv = 10, scoring = 'neg_mean_squared_error')
print(np.sqrt(-scores.mean()))

2.0424952107870706


## Referencias

*   Jacinto, V. R. (2024). Machine learning: Fundamentos, algoritmos y aplicaciones para los negocios, industria y finanzas. Ediciones Díaz de Santos.
*   James, G., Witten, D., Hastie, T., & Tibshirani, R. (2021). An Introduction to Statistical Learning: with Applications in R. https://link.springer.com/content/pdf/10.1007/978-1-0716-1418-1.pdf
*   Student performance (Multiple Linear regression). (2023, June 29). Kaggle. https://www.kaggle.com/datasets/nikhil7280/student-performance-multiple-linear-regression